In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages #reducer function
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver #Legacy name for InMemorySaver
from langgraph.types import interrupt, Command

import yfinance as yf

class State(TypedDict):
    messages: Annotated[list, add_messages]

@tool
def get_stock_price(ticker: str) -> str:
    """
    Get the latest stock price for a given ticker symbol.
    Example tickers: AAPL, GOOGL, TSLA, INFY.NS, TCS.NS, RELIANCE.NS.
    If quantity is mentioned, calculate total price based on the latest stock price and quantity.
    """
    stock = yf.Ticker(ticker)
    data = stock.history(period="1d")

    if data.empty:
        return f"No data found for {ticker}"

    price = data["Close"].iloc[-1]
    return f"The latest price of {ticker} is {price:.2f}"

@tool
def buy_stock(ticker: str, quantity: int) -> str:
    """Request human approval before buying stock."""
    interrupt({
        "action": "buy_stock",
        "ticker": ticker,
        "quantity": quantity
    })
    return "Awaiting approval"

memory = MemorySaver()
configINR = {"configurable": {"thread_id": "INR_Stock_Chatbot"}}

tools = [get_stock_price, buy_stock]
llm = init_chat_model("google_genai:gemini-2.5-flash", temperature=0)
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State) -> State:
    last = state["messages"][-1].content.lower()
    if "buy" in last:
        ai_msg = llm_with_tools.invoke(
            state["messages"],
            tool_choice="buy_stock"
        )
    else:
        ai_msg = llm_with_tools.invoke(state["messages"])
    return {"messages": [ai_msg]}

builder = StateGraph(State)

builder.add_node("chatbot", chatbot)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")
builder.add_edge("chatbot", END)

chatGraph = builder.compile(checkpointer=memory)

response = chatGraph.invoke({"messages": [HumanMessage(content="What is the price of RELIANCE.NS?")]}, config=configINR)
print(response["messages"][-1].content)

response = chatGraph.invoke({"messages": [HumanMessage(content="Buy 10 shares of RELIANCE.NS.")]}, config=configINR)
interrupt_data = response.get("__interrupt__")

if interrupt_data:
    print("⚠️ Approval required")
    print(interrupt_data)

    decision = input("Approve this transaction? (yes/no): ").strip().lower()

    approved = decision in ["yes", "y"]

    if approved:
        interrupt_event = interrupt_data[0]

        ticker = interrupt_event.value["ticker"]
        qty = interrupt_event.value["quantity"]

        response = chatGraph.invoke(
            {
                "messages": [
                    HumanMessage(
                        content=f"Approved. Calculate total cost for {qty} shares of {ticker}."
                    )
                ]
            },
            config=configINR
        )

        print(response["messages"][-1].content)
    else:
        response = chatGraph.invoke(
            {"messages": [HumanMessage(content="Rejected. Cancel the purchase.")]},
            config=configINR
    )

    print(response["messages"][-1].content)

else:
    print(response["messages"][-1].content)


In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages #reducer function
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver #Legacy name for InMemorySaver
from langgraph.types import interrupt, Command

import yfinance as yf

class State(TypedDict):
    messages: Annotated[list, add_messages]

@tool
def get_stock_price(ticker: str) -> str:
    """
    Get the latest stock price for a given ticker symbol.
    Example tickers: AAPL, GOOGL, TSLA, INFY.NS, TCS.NS, RELIANCE.NS.
    If quantity is mentioned, calculate total price based on the latest stock price and quantity.
    """
    stock = yf.Ticker(ticker)
    data = stock.history(period="1d")

    if data.empty:
        return f"No data found for {ticker}"

    price = data["Close"].iloc[-1]
    return f"The latest price of {ticker} is {price:.2f}"

@tool
def buy_stock(ticker: str, quantity: int) -> str:
    """Request human approval before buying stock."""
    interrupt({
        "action": "buy_stock",
        "ticker": ticker,
        "quantity": quantity
    })
    return "Awaiting approval"

memory = MemorySaver()
configINR = {"configurable": {"thread_id": "INR_Stock_Chatbot"}}

tools = [get_stock_price, buy_stock]
llm = init_chat_model("google_genai:gemini-2.5-flash", temperature=0)
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State) -> State:
    last = state["messages"][-1].content.lower()
    if "buy" in last:
        ai_msg = llm_with_tools.invoke(
            state["messages"],
            tool_choice="buy_stock"
        )
    else:
        ai_msg = llm_with_tools.invoke(state["messages"])
    return {"messages": [ai_msg]}

builder = StateGraph(State)

builder.add_node("chatbot", chatbot)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")
builder.add_edge("chatbot", END)

chatGraph = builder.compile(checkpointer=memory)

response = chatGraph.invoke({"messages": [HumanMessage(content="What is the price of AAPL?")]}, config=configINR)
print(response["messages"][-1].content)

response = chatGraph.invoke({"messages": [HumanMessage(content="Buy 10 shares of AAPL")]}, config=configINR)
interrupt_data = response.get("__interrupt__")

if interrupt_data:
    print("⚠️ Approval required")
    print(interrupt_data)

    decision = input("Approve this transaction? (yes/no): ").strip().lower()

    approved = decision in ["yes", "y"]

    if approved:
        interrupt_event = interrupt_data[0]

        ticker = interrupt_event.value["ticker"]
        qty = interrupt_event.value["quantity"]

        response = chatGraph.invoke(
            {
                "messages": [
                    HumanMessage(
                        content=f"Approved. Calculate total cost for {qty} shares of {ticker}."
                    )
                ]
            },
            config=configINR
        )

        print(response["messages"][-1].content)
    else:
        response = chatGraph.invoke(
            {"messages": [HumanMessage(content="Rejected. Cancel the purchase.")]},
            config=configINR
    )

    print(response["messages"][-1].content)

else:
    print(response["messages"][-1].content)
